In [2]:
#Þessi klasi fékkst á þræðinum https://github.com/facebookresearch/fastText/pull/552
#Þar sem Python Api bauð ekki upp á NN greiningu, þá er farinn þessi leið
from fasttext import load_model
import numpy as np
import os
import re
import fasttext
import pandas as pd
import xlsxwriter


class FastTextNN:
    
    def __init__(self, ft_model, ft_matrix=None):
        self.ft_model = ft_model        
        self.ft_words = ft_model.get_words()
        self.word_frequencies = dict(zip(*ft_model.get_words(include_freq=True)))
        self.ft_matrix = ft_matrix
        if self.ft_matrix is None:
            self.ft_matrix = np.empty((len(self.ft_words), ft_model.get_dimension()))
            for i, word in enumerate(self.ft_words):
                self.ft_matrix[i,:] = ft_model.get_word_vector(word)
    
    def find_nearest_neighbor(self, query_word, vectors, n=10,  cossims=None):
        """
        vectors is a 2d numpy array corresponding to the vectors you want to consider

        cossims is a 1d numpy array of size len(vectors), which can be passed for efficiency
        returns the index of the closest n matches to query within vectors and the cosine similarity (cosine the angle between the vectors)

        """
        
        query  = self.ft_model.get_word_vector(query_word)
        if cossims is None:
            cossims = np.matmul(vectors, query, out=cossims)

        norms = np.sqrt((query**2).sum() * (vectors**2).sum(axis=1))
        cossims = cossims/norms
        if query_word in self.ft_words:
            result_i = np.argpartition(-cossims, range(n+1))[1:n+1]
        else:
            result_i = np.argpartition(-cossims, range(n+1))[0:n]
        return list(zip(result_i, cossims[result_i]))

    def nearest_words(self, word, n=10, word_freq=None):
        result = self.find_nearest_neighbor(word, self.ft_matrix, n=n)
        if word_freq:
            return [(self.ft_words[r[0]], round(r[1],3)) for r in result if self.word_frequencies[self.ft_words[r[0]]] >= word_freq]
        else:
            return [(self.ft_words[r[0]], round(r[1],3)) for r in result]

model = load_model('models/igc2018_lemma_sentences_cased_cbow_dim300_epoch20_wngram2_minn3_maxn6.vec')
print('Succsess')

ValueError: models/igc2018_lemma_sentences_cased_cbow_dim300_epoch20_wngram2_minn3_maxn6.vec has wrong file format!

In [11]:
#nota: barn, flóttamaður, hægri, hrun, litur, vitleysa

word = 'litaður'

to_print = {}
fasttext_nn = FastTextNN(model)
results = fasttext_nn.nearest_words(word, n=500)
to_print[word] = results

#print(f'\nThe word "{word}" appears {get_uniq_freq(word)} times in the training \ndata as a uniq word but as a subword it appears {get_freq(word)}')
df = pd.DataFrame.from_dict(to_print)
pd.set_option('display.max_rows', df.shape[0]+1)
print(df)

litaður
0           (ólitaður, 0.891)
1         (upplitaður, 0.849)
2            (Litaður, 0.844)
3         (blálitaður, 0.825)
4         (húðlitaður, 0.815)
5        (rauðlitaður, 0.809)
6          (ljóslitaður, 0.8)
7         (gullitaður, 0.795)
8          (aflitaður, 0.791)
9              (Ýktur, 0.782)
10           (tónaður, 0.779)
11           (eikaður, 0.779)
12         (sveipaður, 0.777)
13           (bitaður, 0.773)
14      (kristallaður, 0.772)
15             (dökkur, 0.77)
16        (límkenndur, 0.768)
17           (málaður, 0.766)
18           (yddaður, 0.763)
19             (ýktur, 0.762)
20       (málmkenndur, 0.761)
21       (gulllitaður, 0.759)
22         (gráleitur, 0.758)
23             (ofinn, 0.757)
24          (Fínlegur, 0.757)
25       (kremlitaður, 0.757)
26         (skásettur, 0.756)
27      (tætingslegur, 0.755)
28          (Slípaður, 0.755)
29      (koparlitaður, 0.755)
30           (gylltur, 0.754)
31           (dulegur, 0.752)
32          (blekaður, 0.752)
33